In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!mv rows.csv\?accessType\=DOWNLOAD reported_crimes.csv

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [812 kB]
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [36.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backpo

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()

# Data Cleaning


In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [0]:
os.listdir()

['.config',
 'spark-warehouse',
 'spark-2.3.1-bin-hadoop2.7',
 'spark-2.3.1-bin-hadoop2.7.tgz',
 'reported_crimes.csv',
 'sample_data']

In [0]:
df=spark.read.csv('reported_crimes.csv',header=True)

In [0]:
import pyspark.sql.functions as f

In [0]:
df=df.withColumn('Date',f.to_date(f.col('Date'),format='MM/dd/yyyy hh:mm:ss a'))

In [0]:
df.count()

7065270

In [0]:
df.cache().count()

7065270

In [0]:
df.show(2)

+--------+-----------+----------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|      Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+----------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        null|2001|08/05/2017 03:

In [0]:
df=df.orderBy('Date',ascending=True)

In [0]:
df.show(2)

+--------+-----------+----------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|      Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+----------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|10389372|   HZ126025|2001-01-01|074XX N CALIFORNI...|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           APARTMENT| false|   false|2411|     024|  50|             2|      11|        null|        null|2001|01/30/2016

In [0]:
sampled_time=df.groupBy('Date').count()

In [0]:
sampled_time.cache().count()

6971

In [0]:
sampled_time.show(2)

+----------+-----+
|      Date|count|
+----------+-----+
|2001-01-01| 1819|
|2001-01-02| 1143|
+----------+-----+
only showing top 2 rows



In [0]:
import pyspark.sql.functions as F

In [0]:
sampled_time=sampled_time.withColumn('split_date',F.split('Date','-'))

In [0]:
sampled_time.show(2,truncate=False)

+----------+-----+--------------+
|Date      |count|split_date    |
+----------+-----+--------------+
|2001-01-01|1819 |[2001, 01, 01]|
|2001-01-02|1143 |[2001, 01, 02]|
+----------+-----+--------------+
only showing top 2 rows



In [0]:
from pyspark.sql.types import IntegerType

In [0]:
sampled_time=sampled_time.withColumn('year',f.col('split_date').getItem(0).cast(IntegerType()))
sampled_time=sampled_time.withColumn('month',f.col('split_date').getItem(1).cast(IntegerType()))
sampled_time=sampled_time.withColumn('day',f.col('split_date').getItem(2).cast(IntegerType()))

In [0]:
sampled_time.show(10)

+----------+-----+--------------+----+-----+---+
|      Date|count|    split_date|year|month|day|
+----------+-----+--------------+----+-----+---+
|2001-01-01| 1819|[2001, 01, 01]|2001|    1|  1|
|2001-01-02| 1143|[2001, 01, 02]|2001|    1|  2|
|2001-01-03| 1151|[2001, 01, 03]|2001|    1|  3|
|2001-01-04| 1166|[2001, 01, 04]|2001|    1|  4|
|2001-01-05| 1267|[2001, 01, 05]|2001|    1|  5|
|2001-01-06| 1290|[2001, 01, 06]|2001|    1|  6|
|2001-01-07| 1104|[2001, 01, 07]|2001|    1|  7|
|2001-01-08| 1157|[2001, 01, 08]|2001|    1|  8|
|2001-01-09| 1184|[2001, 01, 09]|2001|    1|  9|
|2001-01-10| 1237|[2001, 01, 10]|2001|    1| 10|
+----------+-----+--------------+----+-----+---+
only showing top 10 rows



In [0]:
sampled_time=sampled_time.drop('split_date')

In [0]:
sampled_time.show(3)

+----------+-----+----+-----+---+
|      Date|count|year|month|day|
+----------+-----+----+-----+---+
|2001-01-01| 1819|2001|    1|  1|
|2001-01-02| 1143|2001|    1|  2|
|2001-01-03| 1151|2001|    1|  3|
+----------+-----+----+-----+---+
only showing top 3 rows



In [0]:
sample=sampled_time.withColumnRenamed('count','crimes')

In [0]:
sample.show(2)

+----------+------+----+-----+---+
|      Date|crimes|year|month|day|
+----------+------+----+-----+---+
|2001-01-01|  1819|2001|    1|  1|
|2001-01-02|  1143|2001|    1|  2|
+----------+------+----+-----+---+
only showing top 2 rows



In [0]:
monthly=sample.groupby('month').sum('crimes')

In [0]:
monthly.show()

+-----+-----------+
|month|sum(crimes)|
+-----+-----------+
|   12|     525223|
|    1|     567420|
|    6|     623852|
|    3|     578047|
|    5|     626840|
|    9|     608055|
|    4|     580194|
|    8|     648952|
|    7|     655551|
|   10|     614625|
|   11|     553419|
|    2|     483092|
+-----+-----------+



In [0]:
monthly=monthly.withColumnRenamed('sum(crimes)','crimes')

In [0]:
monthly.orderBy('crimes',ascending=False).show()# crimes per month

+-----+------+
|month|crimes|
+-----+------+
|    7|655551|
|    8|648952|
|    5|626840|
|    6|623852|
|   10|614625|
|    9|608055|
|    4|580194|
|    3|578047|
|    1|567420|
|   11|553419|
|   12|525223|
|    2|483092|
+-----+------+



In [0]:
sample.show(2)

+----------+------+----+-----+---+
|      Date|crimes|year|month|day|
+----------+------+----+-----+---+
|2001-01-01|  1819|2001|    1|  1|
|2001-01-02|  1143|2001|    1|  2|
+----------+------+----+-----+---+
only showing top 2 rows



In [0]:
sample.printSchema()

root
 |-- Date: date (nullable = true)
 |-- crimes: long (nullable = false)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



# Window Functions

In [0]:
sample.createOrReplaceTempView('crime_data')

In [0]:
spark.sql('select Date,crimes, sum(crimes) over(order by Date) as running_total from crime_data').show()

+----------+------+-------------+
|      Date|crimes|running_total|
+----------+------+-------------+
|2001-01-01|  1819|         1819|
|2001-01-02|  1143|         2962|
|2001-01-03|  1151|         4113|
|2001-01-04|  1166|         5279|
|2001-01-05|  1267|         6546|
|2001-01-06|  1290|         7836|
|2001-01-07|  1104|         8940|
|2001-01-08|  1157|        10097|
|2001-01-09|  1184|        11281|
|2001-01-10|  1237|        12518|
|2001-01-11|  1228|        13746|
|2001-01-12|  1353|        15099|
|2001-01-13|  1310|        16409|
|2001-01-14|  1118|        17527|
|2001-01-15|  1312|        18839|
|2001-01-16|  1251|        20090|
|2001-01-17|  1288|        21378|
|2001-01-18|  1278|        22656|
|2001-01-19|  1214|        23870|
|2001-01-20|  1256|        25126|
+----------+------+-------------+
only showing top 20 rows



In [0]:
spark.sql('select month,Date,crimes, sum(crimes) over(partition by month order by date) as running_total from crime_data where month==11').show(3)

+-----+----------+------+-------------+
|month|      Date|crimes|running_total|
+-----+----------+------+-------------+
|   11|2001-11-01|  1736|         1736|
|   11|2001-11-02|  1460|         3196|
|   11|2001-11-03|  1354|         4550|
+-----+----------+------+-------------+
only showing top 3 rows



In [0]:
spark.sql('select month,Date,crimes, sum(crimes) over(partition by month order by Date) as running_total from crime_data where month==12').show(3)

+-----+----------+------+-------------+
|month|      Date|crimes|running_total|
+-----+----------+------+-------------+
|   12|2001-12-01|  1435|         1435|
|   12|2001-12-02|  1174|         2609|
|   12|2001-12-03|  1280|         3889|
+-----+----------+------+-------------+
only showing top 3 rows



In [0]:
spark.sql('select month,Date,crimes, sum(crimes) over(partition by month) as running_total from crime_data where month==12').show(3)

+-----+----------+------+-------------+
|month|      Date|crimes|running_total|
+-----+----------+------+-------------+
|   12|2001-12-01|  1435|       525223|
|   12|2001-12-02|  1174|       525223|
|   12|2001-12-03|  1280|       525223|
+-----+----------+------+-------------+
only showing top 3 rows



In [0]:
sample.groupBy('month').sum('crimes').orderBy('sum(crimes)').show()

+-----+-----------+
|month|sum(crimes)|
+-----+-----------+
|    2|     483092|
|   12|     525223|
|   11|     553419|
|    1|     567420|
|    3|     578047|
|    4|     580194|
|    9|     608055|
|   10|     614625|
|    6|     623852|
|    5|     626840|
|    8|     648952|
|    7|     655551|
+-----+-----------+



The `PARTITION BY` clause divides the dataset into partitions based on distinct column values and the `ORDER` BY clause then orders values within those partitions.
The `OVER` clause will run on each partition independantly.
This partition is not the same as partitions in the database made in spark to send to worker nodes in a cluster.

In [0]:
spark.sql('select month,Date,crimes, avg(crimes) over(partition by month order by Date) as average from crime_data where month==12').show(3)

+-----+----------+------+------------------+
|month|      Date|crimes|           average|
+-----+----------+------+------------------+
|   12|2001-12-01|  1435|            1435.0|
|   12|2001-12-02|  1174|            1304.5|
|   12|2001-12-03|  1280|1296.3333333333333|
+-----+----------+------+------------------+
only showing top 3 rows



In [0]:
spark.sql('select month,Date,crimes, avg(crimes) over(partition by month order by Date) as average from crime_data where month==11').show(3)

+-----+----------+------+------------------+
|month|      Date|crimes|           average|
+-----+----------+------+------------------+
|   11|2001-11-01|  1736|            1736.0|
|   11|2001-11-02|  1460|            1598.0|
|   11|2001-11-03|  1354|1516.6666666666667|
+-----+----------+------+------------------+
only showing top 3 rows



In [0]:
spark.sql('select month,Date,crimes, count(crimes) over(partition by month order by Date) as count from crime_data where month==12').show(3)

+-----+----------+------+-----+
|month|      Date|crimes|count|
+-----+----------+------+-----+
|   12|2001-12-01|  1435|    1|
|   12|2001-12-02|  1174|    2|
|   12|2001-12-03|  1280|    3|
+-----+----------+------+-----+
only showing top 3 rows



In [0]:
spark.sql('select month,Date,crimes, count(crimes) over(partition by month order by Date) as count from crime_data where month==11').show(3)

+-----+----------+------+-----+
|month|      Date|crimes|count|
+-----+----------+------+-----+
|   11|2001-11-01|  1736|    1|
|   11|2001-11-02|  1460|    2|
|   11|2001-11-03|  1354|    3|
+-----+----------+------+-----+
only showing top 3 rows



In [0]:
spark.sql('select month,Date,crimes, dense_rank() over(partition by month order by Date) as row from crime_data where (month==12 or month==11) and year==2001 and day<=3').show()

+-----+----------+------+---+
|month|      Date|crimes|row|
+-----+----------+------+---+
|   12|2001-12-01|  1435|  1|
|   12|2001-12-02|  1174|  2|
|   12|2001-12-03|  1280|  3|
|   11|2001-11-01|  1736|  1|
|   11|2001-11-02|  1460|  2|
|   11|2001-11-03|  1354|  3|
+-----+----------+------+---+



In [0]:
spark.sql('select month,Date,crimes, ntile(4) over(partition by month order by Date) as row from crime_data where (month==12 or month==11) and year==2001 and day<=3').show()

+-----+----------+------+---+
|month|      Date|crimes|row|
+-----+----------+------+---+
|   12|2001-12-01|  1435|  1|
|   12|2001-12-02|  1174|  2|
|   12|2001-12-03|  1280|  3|
|   11|2001-11-01|  1736|  1|
|   11|2001-11-02|  1460|  2|
|   11|2001-11-03|  1354|  3|
+-----+----------+------+---+



Here in ntile(n) n is the number of buckets that the table has been divided into.

In [0]:
spark.sql('select month,Date,crimes,ntile(2)over(partition by month order by Date) as percentile from crime_data where (month == 12 or month == 11) and year==2001 and day<=3').show()

+-----+----------+------+----------+
|month|      Date|crimes|percentile|
+-----+----------+------+----------+
|   12|2001-12-01|  1435|         1|
|   12|2001-12-02|  1174|         1|
|   12|2001-12-03|  1280|         2|
|   11|2001-11-01|  1736|         1|
|   11|2001-11-02|  1460|         1|
|   11|2001-11-03|  1354|         2|
+-----+----------+------+----------+



In [0]:
pct_chg=spark.sql("select month,Date,day,crimes,lag(crimes,3)over(partition by month order by Date)as threelag from crime_data where year==2001 and month>=6 and day<=10 order by month asc limit 20")

In [0]:
pct_chg.show(20)

+-----+----------+---+------+--------+
|month|      Date|day|crimes|threelag|
+-----+----------+---+------+--------+
|    6|2001-06-01|  1|  1662|    null|
|    6|2001-06-02|  2|  1398|    null|
|    6|2001-06-03|  3|  1184|    null|
|    6|2001-06-04|  4|  1371|    1662|
|    6|2001-06-05|  5|  1281|    1398|
|    6|2001-06-06|  6|  1344|    1184|
|    6|2001-06-07|  7|  1326|    1371|
|    6|2001-06-08|  8|  1370|    1281|
|    6|2001-06-09|  9|  1386|    1344|
|    6|2001-06-10| 10|  1413|    1326|
|    7|2001-07-01|  1|  1601|    null|
|    7|2001-07-02|  2|  1372|    null|
|    7|2001-07-03|  3|  1316|    null|
|    7|2001-07-04|  4|  1420|    1601|
|    7|2001-07-05|  5|  1375|    1372|
|    7|2001-07-06|  6|  1455|    1316|
|    7|2001-07-07|  7|  1447|    1420|
|    7|2001-07-08|  8|  1385|    1375|
|    7|2001-07-09|  9|  1431|    1455|
|    7|2001-07-10| 10|  1492|    1447|
+-----+----------+---+------+--------+



In [0]:
pct_chg=pct_chg.fillna({'threelag':0})

In [0]:
pct_chg.show(10)

+-----+----------+---+------+--------+
|month|      Date|day|crimes|threelag|
+-----+----------+---+------+--------+
|    6|2001-06-01|  1|  1662|       0|
|    6|2001-06-02|  2|  1398|       0|
|    6|2001-06-03|  3|  1184|       0|
|    6|2001-06-04|  4|  1371|    1662|
|    6|2001-06-05|  5|  1281|    1398|
|    6|2001-06-06|  6|  1344|    1184|
|    6|2001-06-07|  7|  1326|    1371|
|    6|2001-06-08|  8|  1370|    1281|
|    6|2001-06-09|  9|  1386|    1344|
|    6|2001-06-10| 10|  1413|    1326|
+-----+----------+---+------+--------+
only showing top 10 rows



In [0]:
pct_chg=pct_chg.withColumn('percentage change',(((pct_chg.crimes-pct_chg.threelag)/pct_chg.threelag)*100))

In [0]:
pct_chg=pct_chg.na.drop(subset=[col for col in pct_chg.columns])

In [0]:
pct_chg.show(10)

+-----+----------+---+------+--------+-------------------+
|month|      Date|day|crimes|threelag|  percentage change|
+-----+----------+---+------+--------+-------------------+
|    6|2001-06-04|  4|  1371|    1662|-17.509025270758123|
|    6|2001-06-05|  5|  1281|    1398| -8.369098712446352|
|    6|2001-06-06|  6|  1344|    1184| 13.513513513513514|
|    6|2001-06-07|  7|  1326|    1371| -3.282275711159737|
|    6|2001-06-08|  8|  1370|    1281|  6.947697111631538|
|    6|2001-06-09|  9|  1386|    1344|              3.125|
|    6|2001-06-10| 10|  1413|    1326|  6.561085972850679|
|    7|2001-07-04|  4|  1420|    1601|-11.305434103685197|
|    7|2001-07-05|  5|  1375|    1372|0.21865889212827988|
|    7|2001-07-06|  6|  1455|    1316| 10.562310030395135|
+-----+----------+---+------+--------+-------------------+
only showing top 10 rows



In [0]:
from pyspark.sql.window import Window

In [0]:
windowSpec=Window.partitionBy(sample.month).orderBy(sample.Date)

In [0]:
sampled=sample.select('month','Date','crimes',F.lag('crimes',3).over(windowSpec).alias('threelag'),F.row_number().over(windowSpec).alias('row_number')).where('year=2001').orderBy('month')

In [0]:
sampled.cache()

DataFrame[month: int, Date: date, crimes: bigint, threelag: bigint, row_number: int]

In [0]:
sampled.show(10)

+-----+----------+------+--------+----------+
|month|      Date|crimes|threelag|row_number|
+-----+----------+------+--------+----------+
|    1|2001-01-01|  1819|    null|         1|
|    1|2001-01-02|  1143|    null|         2|
|    1|2001-01-03|  1151|    null|         3|
|    1|2001-01-04|  1166|    1819|         4|
|    1|2001-01-05|  1267|    1143|         5|
|    1|2001-01-06|  1290|    1151|         6|
|    1|2001-01-07|  1104|    1166|         7|
|    1|2001-01-08|  1157|    1267|         8|
|    1|2001-01-09|  1184|    1290|         9|
|    1|2001-01-10|  1237|    1104|        10|
+-----+----------+------+--------+----------+
only showing top 10 rows



In [0]:
spark.sql('select month,Date,crimes,avg(crimes)over(partition by month order by Date asc rows between 3 preceding and 3 following) as moving,count(crimes)over(partition by month order by Date asc rows 3 preceding) as counting from crime_data').show()

+-----+----------+------+------------------+--------+
|month|      Date|crimes|            moving|counting|
+-----+----------+------+------------------+--------+
|   12|2001-12-01|  1435|           1316.75|       1|
|   12|2001-12-02|  1174|            1320.4|       2|
|   12|2001-12-03|  1280|1319.1666666666667|       3|
|   12|2001-12-04|  1378|1341.5714285714287|       4|
|   12|2001-12-05|  1335|1316.2857142857142|       4|
|   12|2001-12-06|  1313| 1305.142857142857|       4|
|   12|2001-12-07|  1476|1304.7142857142858|       4|
|   12|2001-12-08|  1258|1289.5714285714287|       4|
|   12|2001-12-09|  1096|1285.2857142857142|       4|
|   12|2001-12-10|  1277|1288.4285714285713|       4|
|   12|2001-12-11|  1272|1273.2857142857142|       4|
|   12|2001-12-12|  1305| 1277.857142857143|       4|
|   12|2001-12-13|  1335|1280.4285714285713|       4|
|   12|2001-12-14|  1370|            1277.0|       4|
|   12|2001-12-15|  1290|            1274.0|       4|
|   12|2001-12-16|  1114| 12

In [0]:
spark.sql('select month,Date,crimes,avg(crimes)over(partition by month order by Date) as moving from crime_data').show()

+-----+----------+------+------------------+
|month|      Date|crimes|            moving|
+-----+----------+------+------------------+
|   12|2001-12-01|  1435|            1435.0|
|   12|2001-12-02|  1174|            1304.5|
|   12|2001-12-03|  1280|1296.3333333333333|
|   12|2001-12-04|  1378|           1316.75|
|   12|2001-12-05|  1335|            1320.4|
|   12|2001-12-06|  1313|1319.1666666666667|
|   12|2001-12-07|  1476|1341.5714285714287|
|   12|2001-12-08|  1258|          1331.125|
|   12|2001-12-09|  1096|            1305.0|
|   12|2001-12-10|  1277|            1302.2|
|   12|2001-12-11|  1272|1299.4545454545455|
|   12|2001-12-12|  1305|1299.9166666666667|
|   12|2001-12-13|  1335|1302.6153846153845|
|   12|2001-12-14|  1370|1307.4285714285713|
|   12|2001-12-15|  1290|1306.2666666666667|
|   12|2001-12-16|  1114|           1294.25|
|   12|2001-12-17|  1253|1291.8235294117646|
|   12|2001-12-18|  1251|1289.5555555555557|
|   12|2001-12-19|  1234|1286.6315789473683|
|   12|200

In [0]:
spark.sql('select month,Date,crimes,avg(crimes)over(partition by month order by Date asc rows between 3 preceding and 3 following) as moving,count(crimes)over(partition by month order by Date asc rows 3 preceding) as counting from crime_data').show()

+-----+----------+------+------------------+--------+
|month|      Date|crimes|            moving|counting|
+-----+----------+------+------------------+--------+
|   12|2001-12-01|  1435|           1316.75|       1|
|   12|2001-12-02|  1174|            1320.4|       2|
|   12|2001-12-03|  1280|1319.1666666666667|       3|
|   12|2001-12-04|  1378|1341.5714285714287|       4|
|   12|2001-12-05|  1335|1316.2857142857142|       4|
|   12|2001-12-06|  1313| 1305.142857142857|       4|
|   12|2001-12-07|  1476|1304.7142857142858|       4|
|   12|2001-12-08|  1258|1289.5714285714287|       4|
|   12|2001-12-09|  1096|1285.2857142857142|       4|
|   12|2001-12-10|  1277|1288.4285714285713|       4|
|   12|2001-12-11|  1272|1273.2857142857142|       4|
|   12|2001-12-12|  1305| 1277.857142857143|       4|
|   12|2001-12-13|  1335|1280.4285714285713|       4|
|   12|2001-12-14|  1370|            1277.0|       4|
|   12|2001-12-15|  1290|            1274.0|       4|
|   12|2001-12-16|  1114| 12

In [0]:
df=spark.sql("select * from crime_data")

In [0]:
df.show(3)

+----------+------+----+-----+---+
|      Date|crimes|year|month|day|
+----------+------+----+-----+---+
|2001-01-01|  1819|2001|    1|  1|
|2001-01-02|  1143|2001|    1|  2|
|2001-01-03|  1151|2001|    1|  3|
+----------+------+----+-----+---+
only showing top 3 rows



In [0]:
from pyspark.sql.window import Window

In [0]:
window=Window.partitionBy('month').orderBy('Date').rowsBetween(-3,0)

In [0]:
df.select(df.month,df.Date,df.crimes,F.avg(df.crimes).over(window).alias('moving'),F.count(df.crimes).over(window).alias('counting')).show()

+-----+----------+------+------------------+--------+
|month|      Date|crimes|            moving|counting|
+-----+----------+------+------------------+--------+
|   12|2001-12-01|  1435|            1435.0|       1|
|   12|2001-12-02|  1174|            1304.5|       2|
|   12|2001-12-03|  1280|1296.3333333333333|       3|
|   12|2001-12-04|  1378|           1316.75|       4|
|   12|2001-12-05|  1335|           1291.75|       4|
|   12|2001-12-06|  1313|            1326.5|       4|
|   12|2001-12-07|  1476|            1375.5|       4|
|   12|2001-12-08|  1258|            1345.5|       4|
|   12|2001-12-09|  1096|           1285.75|       4|
|   12|2001-12-10|  1277|           1276.75|       4|
|   12|2001-12-11|  1272|           1225.75|       4|
|   12|2001-12-12|  1305|            1237.5|       4|
|   12|2001-12-13|  1335|           1297.25|       4|
|   12|2001-12-14|  1370|            1320.5|       4|
|   12|2001-12-15|  1290|            1325.0|       4|
|   12|2001-12-16|  1114|   

In [0]:
spark.sql('select month,Date,crimes,avg(crimes)over(partition by month order by Date asc rows between 3 preceding and current row) as moving,count(crimes)over(partition by month order by Date asc rows 3 preceding) as counting from crime_data').show()

+-----+----------+------+------------------+--------+
|month|      Date|crimes|            moving|counting|
+-----+----------+------+------------------+--------+
|   12|2001-12-01|  1435|            1435.0|       1|
|   12|2001-12-02|  1174|            1304.5|       2|
|   12|2001-12-03|  1280|1296.3333333333333|       3|
|   12|2001-12-04|  1378|           1316.75|       4|
|   12|2001-12-05|  1335|           1291.75|       4|
|   12|2001-12-06|  1313|            1326.5|       4|
|   12|2001-12-07|  1476|            1375.5|       4|
|   12|2001-12-08|  1258|            1345.5|       4|
|   12|2001-12-09|  1096|           1285.75|       4|
|   12|2001-12-10|  1277|           1276.75|       4|
|   12|2001-12-11|  1272|           1225.75|       4|
|   12|2001-12-12|  1305|            1237.5|       4|
|   12|2001-12-13|  1335|           1297.25|       4|
|   12|2001-12-14|  1370|            1320.5|       4|
|   12|2001-12-15|  1290|            1325.0|       4|
|   12|2001-12-16|  1114|   

In [0]:
df=df.select(df.month,df.Date,df.crimes,F.avg(df.crimes).over(window).alias('moving'))

In [0]:
df.cache()

DataFrame[month: int, Date: date, crimes: bigint, moving: double, counting: bigint]

In [0]:
df.show(5)

+-----+----------+------+------------------+--------+
|month|      Date|crimes|            moving|counting|
+-----+----------+------+------------------+--------+
|   12|2001-12-01|  1435|            1435.0|       1|
|   12|2001-12-02|  1174|            1304.5|       2|
|   12|2001-12-03|  1280|1296.3333333333333|       3|
|   12|2001-12-04|  1378|           1316.75|       4|
|   12|2001-12-05|  1335|           1291.75|       4|
+-----+----------+------+------------------+--------+
only showing top 5 rows



In [0]:
df=df.select(df.month,df.Date,df.crimes,F.avg(df.crimes).over(window).alias('moving'))
window=Window.partitionBy('month').orderBy('Date').rowsBetween(-3,0)

In [0]:
sample.show(2)

+----------+------+----+-----+---+
|      Date|crimes|year|month|day|
+----------+------+----+-----+---+
|2001-01-01|  1819|2001|    1|  1|
|2001-01-02|  1143|2001|    1|  2|
+----------+------+----+-----+---+
only showing top 2 rows



#Simple Window


In [0]:
def simpleWindow(df,alias,apply_to='',partition_by=None,order_by=None,action='count',preceding=0,following=0):
  '''
  Apply window operations on dataframe and adds to df with a new column.
  Returns: PySpark DataFrame
  Parameters:
  * df : PySpark DataFrame 
  * alias : Name of new column with results of window operation
  * apply_to : Column to which window operation on which window operation has to be applied
  * partition_by : Column whose values will be used to partition the dataset
  * order_by : Column whose values will be used to order the values of each partition
  * action : 'count','average','sum','rank','ntile','rank','dense_rank'
  * preceding : rows before current row
  * following : rows after current row  
  '''
  from pyspark.sql.window import Window
  import pyspark.sql.functions as F
  if partition_by and order_by:
    someWindow=Window.partitionBy(partition_by).orderBy(order_by).rowsBetween(preceding,following)
  elif partition_by:
    someWindow=Window.partitionBy(partition_by).rowsBetween(preceding,following)
  elif order_by:
    someWindow=Window.orderBy(order_by).rowsBetween(preceding,following)
  else:
    raise(Exception('Cannot keep both partition_by and order_by undefined.'))
  actiondict={
      'count':F.count,
      'average':F.avg,
      'sum':F.sum,
      'ntile':F.ntile,
      'rank':F.rank,
      'dense_rank':F.dense_rank
      }
  specialfunc=actiondict[action]
  return df.withColumn(alias,specialfunc(apply_to).over(someWindow))

In [0]:
samplex=betterWindow(sample,'counting',apply_to='crimes',partition_by='month',order_by='Date',action='count',preceding=-3,following=0)

In [0]:
samplex.show()

+----------+------+----+-----+---+--------+
|      Date|crimes|year|month|day|counting|
+----------+------+----+-----+---+--------+
|2001-12-01|  1435|2001|   12|  1|       1|
|2001-12-02|  1174|2001|   12|  2|       2|
|2001-12-03|  1280|2001|   12|  3|       3|
|2001-12-04|  1378|2001|   12|  4|       4|
|2001-12-05|  1335|2001|   12|  5|       4|
|2001-12-06|  1313|2001|   12|  6|       4|
|2001-12-07|  1476|2001|   12|  7|       4|
|2001-12-08|  1258|2001|   12|  8|       4|
|2001-12-09|  1096|2001|   12|  9|       4|
|2001-12-10|  1277|2001|   12| 10|       4|
|2001-12-11|  1272|2001|   12| 11|       4|
|2001-12-12|  1305|2001|   12| 12|       4|
|2001-12-13|  1335|2001|   12| 13|       4|
|2001-12-14|  1370|2001|   12| 14|       4|
|2001-12-15|  1290|2001|   12| 15|       4|
|2001-12-16|  1114|2001|   12| 16|       4|
|2001-12-17|  1253|2001|   12| 17|       4|
|2001-12-18|  1251|2001|   12| 18|       4|
|2001-12-19|  1234|2001|   12| 19|       4|
|2001-12-20|  1174|2001|   12| 2

In [0]:
help(simpleWindow)

Help on function simpleWindow in module __main__:

simpleWindow(df, alias, apply_to='', partition_by=None, order_by=None, action='count', preceding=0, following=0)
    Apply window operations on dataframe and adds to df with a new column.
    Returns: PySpark DataFrame
    Parameters:
    * df : PySpark DataFrame 
    * alias : Name of new column with results of window operation
    * apply_to : Column to which window operation on which window operation has to be applied
    * partition_by : Column whose values will be used to partition the dataset
    * order_by : Column whose values will be used to order the values of each partition
    * action : 'count','average','sum','rank','ntile','rank','dense_rank'
    * preceding : rows before current row
    * following : rows after current row

